## Instala dependências

In [21]:
pip install apyori

## Importa os arquivos .csv

In [23]:
import pandas as pd
from apyori import apriori

# Carregar os arquivos CSV
try:
    cardapio = pd.read_csv("tabelas/cardapio.csv")
    transacoes = pd.read_csv("tabelas/transacoes.csv")
    print("Arquivos carregados com sucesso!")
except FileNotFoundError:
    print("Erro: Um ou ambos os arquivos CSV não foram encontrados. Certifique-se de que os arquivos estão no mesmo diretório do notebook ou forneça o caminho correto.")
except Exception as e:
    print(f"Ocorreu um erro ao carregar os arquivos: {e}")

if 'cardapio' in locals():
    print("\nPrimeiras linhas do cardapio.csv:")
    print(cardapio.head())

if 'transacoes' in locals():
    print("\nPrimeiras linhas do transacoes.csv:")
    print(transacoes.head())

Arquivos carregados com sucesso!

Primeiras linhas do cardapio.csv:
   id_produto          nome_produto   categoria  preco
0           1          Neural Smash  Hamburguer   26.9
1           2        AI Bacon Boost  Hamburguer   28.9
2           3  Deep Learning Double  Hamburguer   32.9
3           4         Quantum Blend  Hamburguer   34.9
4           5         Cyber Classic  Hamburguer   30.9

Primeiras linhas do transacoes.csv:
   id_transacao  id_cliente  id_produto
0             1           1          12
1             1           1           9
2             2           3          18
3             2           3          14
4             2           3           4


## Agrupa os produtos por pedido

In [24]:
# Agrupar os produtos por ID de transação e criar uma lista de listas
transacoes_agrupadas = transacoes.groupby('id_transacao')['id_produto'].apply(list).tolist()

print("Primeiras transações agrupadas:")
print(transacoes_agrupadas[:5])

Primeiras transações agrupadas:
[[12, 9], [18, 14, 4, 10], [8, 15, 7, 1, 8], [13, 5], [2, 9, 12, 15]]


## Aplica o Apriori de acordo com as regras definidas

In [28]:
# Aplicar o algoritmo Apriori
regras = apriori(transacoes_agrupadas, min_support=0.03, min_confidence=0.2, min_lift=1.5)

resultados = list(regras)

print(f"Número de regras de associação encontradas: {len(resultados)}")

for item in resultados[:3]:
    pair = item[0]
    items = [x for x in pair]
    print(f"Itemset base: {items}")

    for relate in item[2]:
        print(f"  Regra: {relate.items_base} -> {relate.items_add}")
        print(f"  Suporte: {item[1]:.4f}")
        print(f"  Confiança: {relate.confidence:.4f}")
        print(f"  Lift: {relate.lift:.4f}")
    print("---")

Número de regras de associação encontradas: 32
Itemset base: [8, 1]
  Regra: frozenset({1}) -> frozenset({8})
  Suporte: 0.1085
  Confiança: 0.5306
  Lift: 2.2968
  Regra: frozenset({8}) -> frozenset({1})
  Suporte: 0.1085
  Confiança: 0.4697
  Lift: 2.2968
---
Itemset base: [9, 2]
  Regra: frozenset({2}) -> frozenset({9})
  Suporte: 0.0570
  Confiança: 0.3775
  Lift: 1.6593
  Regra: frozenset({9}) -> frozenset({2})
  Suporte: 0.0570
  Confiança: 0.2505
  Lift: 1.6593
---
Itemset base: [2, 12]
  Regra: frozenset({2}) -> frozenset({12})
  Suporte: 0.0570
  Confiança: 0.3775
  Lift: 1.7598
  Regra: frozenset({12}) -> frozenset({2})
  Suporte: 0.0570
  Confiança: 0.2657
  Lift: 1.7598
---


# Recebe Input e realiza a predição

In [29]:
# Função que recebe entrada para o Apriori
def recomendar_produtos(carrinho, regras, top_n=3):
    recomendacoes = {}
    carrinho_frozenset = frozenset(carrinho)
    print("---")
    print(f"\nAnalisando recomendações para o carrinho: {carrinho}")
    for item in regras:
        pair = item[0]
        for relate in item[2]:
            items_base = relate.items_base
            items_add = relate.items_add
            produto_recomendado = next(iter(items_add))

            if items_base.issubset(carrinho_frozenset):
                if produto_recomendado not in carrinho:
                    if produto_recomendado not in recomendacoes or relate.lift > recomendacoes[produto_recomendado]['lift']:
                        recomendacoes[produto_recomendado] = {'confidence': relate.confidence, 'lift': relate.lift}
                        print(f"    Recomendação potencial adicionada: {produto_recomendado} (Lift: {relate.lift:.4f})")


    recomendacoes_ordenadas = sorted(recomendacoes.items(), key=lambda item: item[1]['lift'], reverse=True)
    return [item[0] for item in recomendacoes_ordenadas[:top_n]]

# Testes de entradas
carrinho_exemplo = [1, 7]
recomendacoes = recomendar_produtos(carrinho_exemplo, resultados)
print(f"Recomendações para {carrinho_exemplo}: {recomendacoes}")

carrinho_exemplo_2 = [9]
recomendacoes_2 = recomendar_produtos(carrinho_exemplo_2, resultados)
print(f"Recomendações para {carrinho_exemplo_2}: {recomendacoes_2}")

carrinho_exemplo_3 = [2, 12]
recomendacoes_3 = recomendar_produtos(carrinho_exemplo_3, resultados)
print(f"Recomendações para {carrinho_exemplo_3}: {recomendacoes_3}")

---

Analisando recomendações para o carrinho: [1]
    Recomendação potencial adicionada: 8 (Lift: 2.2968)
    Recomendação potencial adicionada: 8 (Lift: 3.1213)
Recomendações para [1]: [8]
---

Analisando recomendações para o carrinho: [9]
    Recomendação potencial adicionada: 2 (Lift: 1.6593)
    Recomendação potencial adicionada: 3 (Lift: 1.9100)
    Recomendação potencial adicionada: 12 (Lift: 2.1312)
    Recomendação potencial adicionada: 15 (Lift: 1.8499)
    Recomendação potencial adicionada: 12 (Lift: 3.0924)
Recomendações para [9]: [12, 3, 15]
---

Analisando recomendações para o carrinho: [2, 12]
    Recomendação potencial adicionada: 9 (Lift: 1.6593)
    Recomendação potencial adicionada: 3 (Lift: 1.8593)
    Recomendação potencial adicionada: 9 (Lift: 2.1312)
    Recomendação potencial adicionada: 15 (Lift: 1.7910)
    Recomendação potencial adicionada: 9 (Lift: 2.9939)
Recomendações para [2, 12]: [9, 3, 15]


In [30]:
pd.set_option('display.max_columns', None) # coluna
pd.set_option('display.max_rows', None)

display(cardapio)

,id_produto,nome_produto,categoria,preco
0,1,Neural Smash,Hamburguer,26.9
1,2,AI Bacon Boost,Hamburguer,28.9
2,3,Deep Learning Double,Hamburguer,32.9
3,4,Quantum Blend,Hamburguer,34.9
4,5,Cyber Classic,Hamburguer,30.9
5,6,Data Crunch,Hamburguer,33.9
6,7,Machine Melt,Hamburguer,31.9
7,8,Batata Frita Clássica,Acompanhamento,12.9
8,9,Batata Frita com Cheddar e Bacon,Acompanhamento,19.9
9,10,Onion Rings,Acompanhamento,14.9
